## Initialization

In [41]:
import sys
#sys.path.append('/Users/chandrasekharmukherjee/Home/Centrality framework/Modules')
sys.path.append('I:/내 드라이브/backup/document/USC/Research/MCPC/Code/Codes')

#Import packages
#imports
import time
import pandas as pd
from collections import Counter
import sknetwork


from sknetwork.ranking import PageRank
from sknetwork.ranking import Betweenness
from sknetwork.ranking import Closeness

#import umap as umap

from numba.typed import List
import warnings
from numba import njit
import pynndescent
import numpy as np
from sklearn.cluster import SpectralClustering
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib
import operator
from sklearn.utils.extmath import randomized_svd
from random import randint
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
import scipy
#from umap.umap_ import *
import math
from random import randint
import keras
from keras.datasets import mnist
from keras.datasets import cifar10
from keras.datasets import cifar100
from keras.datasets import fashion_mnist
import scanpy
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import adjusted_mutual_info_score, roc_auc_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score

#import community as community_louvain
from sklearn.cluster import KMeans
from sklearn.metrics import fowlkes_mallows_score

import scipy

import igraph 
import networkx as nx


import metric as met 
import embedding as embed   
import data_utils_ch as data_util

%load_ext autoreload
%autoreload 2
import importlib

import metric as met 
import embedding as embed   
import data_utils_ch as data_util
from subgraph import *


met=importlib.reload(met)
embed=importlib.reload(embed)
data_util=importlib.reload(data_util)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Testing every dataset with top k % nodes induced subgraph (Louvain)

### Top k% induced subgraph + Strong Majority Vote later

##### PDF Export version (Final Working Version)

In [40]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import time 
import debug

res_=[] #0.26, 0.24 is best for Louvain, 3.3, 2.0
labels=[]
total_labels=[]
names = []

data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['Zhengmix8eq']
'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''
datanames = ['Cora full', 'Citeseer', 'Eu core']
# FR_ord = ['Rd', 'Ord']
# FR_Recalc = ['FR_avg', 'FR_Re']
FR_tp = ['FL','FL_ng','FL_ng_prop','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}
Mod_tp = {0: 'Louv'}

#top k% nodes for induced subgraph
k=0.2
traverse = {0:'Reset',1:'Static'}
ord = {0:'Rand',1:'FR_order'}

#list of plotted figures
figures = []

for data in data_names: 
    pdf_name = './Plots/'+data+'/fixed_'+data+'.pdf'
    
    for survive in [0]:
        '''
        Get the Graph from Edge_List and get the labels
        '''
        #DO not recalculate Graph for non bulk RNA datasets
        if data not in ['mRNA','miRNA'] and survive==1:
            continue
        G, label= data_to_graph(data,survive)
        #add self loop to nodes without outgoing edges
        #G_ is copy of G
        G_ = G.copy()
        if (data in ['Cora full', 'Citeseer', 'Eu core']):
            for node in G_.nodes():
                if not list(G_.successors(node)):
                    G_.remove_node(node)
        # for node in G_.nodes():
        #     if not list(G_.successors(node)):
        #         print(data, ' has a node without outgoing edges:',node)
        #         G_.add_edge(node, node, weight=1)
        print('Number of nodes:',G.number_of_nodes())
        

        data_directory = f'./Plots/{data}'
        file_path = f'{data_directory}/{data}.txt'
        os.makedirs(data_directory, exist_ok=True)
        with open(file_path, 'a') as f:
            f.write('------------------------------------------------------------------------\n')
        
        
            
        for Num_hop in [1]:
            plt.figure(figsize=(10,15)) # figure for NMI
            plt.figure(figsize=(10,15)) # figure for purity
            plt.figure(figsize=(10,15)) # figure for Incoming Edge Degrees
            #List of FR_values
            FR_precalculated = []
            for fr_type in range(4): 
                '''
                Calculate FlowRank for the whole graph
                '''
                if Num_hop==0:
                    node2FR = calc_FlowRank(G_, fr_type, 5)
                else:
                    node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
                FR_precalculated.append(node2FR)

            for idx, res in enumerate([0.05,0.25, 0.5, 1, 1.5, 5]):
            #for idx, res in enumerate([0.01,0.03,0.05,0.1,0.15,0.2]):
                for Mod_type in [0]:
                    ''' 
                    Louvain on G (baseline)
                    '''
                    #start_time = time.time()
                    partition = debug.louvain_partitions(G, seed=0,resolution=res)
                    partition_ = deque(partition, maxlen=1).pop()
                    label_ = part_to_full_label(partition_,G.number_of_nodes())
                    NMI_baseline = NMI(label_, label)
                    Purity_baseline = met.purity_score(label, label_)
                    
                    #print('Time Louvain on whole G:', time.time()-start_time)
                    for FR_type in range(4):
                        name =  data + ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops'
                        if FR_type==3:
                            name = data + ' (top ' + str(k*100) + '%): PageRank'
                        if data in ['mRNA','miRNA']:
                            name = data +  ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops' + ' + Survive: '+str(survive)
                            if FR_type==3:
                                name = data + ' (top ' + str(k*100) + '%): PageRank' + ' + Survive: '+str(survive)
                        print(' ---',name,'| res: ',str(res))
                        
                        # Create the directory if it doesn't exist
                        os.makedirs(data_directory, exist_ok=True)
                        with open(file_path, 'a') as f:
                            f.write(' ---'+name+'| res: '+str(res)+'\n')
                            f.write('NMI baseline: ' + str(round(NMI_baseline,3)) + '\n')
                        

                        '''
                        Get induced subgraph of top k% nodes in FR value
                        '''
                        node2FR = FR_precalculated[FR_type]
                        
                        if data in ['Cora full', 'Citeseer', 'Eu core']:
                            for node in G.nodes():
                                if not list(G.successors(node)):
                                    node2FR[node]=0

                        node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                        #print('Time sort:', time.time()-start_time)
                        
                        k_ = int(k*len(node_list))
                        top_nodes = node_list[:k_]
                        H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                        #print('Time Induced Subgraph:', time.time()-start_time)
                        
                        

                        '''
                        Louvain on the subgraph H 
                        '''
                        # partition = RC10.louvain_partitions(H, seed=0,resolution=res, FR_type=FR_type, Mod_type=Mod_type)
                        #partition = Louv.louvain_partitions(H, seed=0,resolution=res ,threshold=0.0001)
                        
                        partition = debug.louvain_partitions(H, seed=0,resolution=res)
                        partition_ = deque(partition, maxlen=1).pop()
                        H_label = part_to_full_label(partition_,G.number_of_nodes())
                        #print('Time Louvain:', time.time()-start_time)
        
                        '''Run Louvain Again on the rest ( Warm Start )'''
                        #start_time = time.time()

                        k_ = int(k*len(node_list))
                        nodes_rest = node_list[k_:]
                        
                        for node in nodes_rest:
                            partition_.append({node})
                        part = Cust.louvain_partitions(G, seed=0,resolution=res,init_part=partition_)
                        part_ = deque(part, maxlen=1).pop()
                        label_ = part_to_full_label(part_,G.number_of_nodes())
                        NMI_twice = NMI(label_, label)
                        Purity_twice = met.purity_score(label, label_)
                        #print('Time Louvain twice:', time.time()-start_time)
                        print('NMI baseline:', round(NMI_baseline,3))
                        print('NMI Louvain twice:', round(NMI_twice,3))


                        '''
                        Add nodes to the subgraph by strong majority vote
                        '''
                        #See the trend by adding node to the subgraph by strong majority vote (in order of FlowRank Value)
                        
                        #print('H_label: ',H_label, 'len: ',len(H_label))
                        
                        
                        #start_time = time.time()
                        NMI_List, Purity_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label)
                        #print('Time merge by vote:', time.time()-start_time)
                        
                        #Plot NMI_List
                        fig_numbers = plt.get_fignums()
                        nmi_fig =fig_numbers[-3]
                        purity_fig = fig_numbers[-2]
                        InEdge_fig = fig_numbers[-1]

                        #plot NMI
                        plt.figure(nmi_fig)
                        plt.subplot(3, 2, idx+1)
                        if FR_type==0: 
                            #Horizontal Line for Baselines
                            NMI_baseline_list = [NMI_baseline]*len(nodes_rest)
                            plt.plot(NMI_baseline_list, label = 'Louv_baseline')
                    
                        NMI_Louv_twice_list = [NMI_twice]*len(nodes_rest)
                        plt.plot(NMI_Louv_twice_list, label = '2*Louv|'+str(FR_tp_short[FR_type]))
                        plt.plot(NMI_List, label = str(FR_tp[FR_type]))
                        with open(file_path, 'a') as f:
                            f.write('# of nodes not included: ' + str(G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) + '\n')
                            f.write('NMI_Louvain_twice: ' + str(round(NMI_twice,3)) + '\n')
                            f.write('NMI after top ' + str(k*100) + '% nodes: ' + str(round(NMI_List[0],3)) + '\n')
                            f.write('Max NMI: ' + str(round(max(NMI_List),3)) + '\n')
                        print('Number of nodes not included: ' , G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) 
                        print('NMI after top ', k*100, '% nodes:', round(NMI_List[0],3))
                        print('Max NMI: ', round(max(NMI_List),3))

                        #plot Purity
                        plt.figure(purity_fig)
                        plt.subplot(3, 2, idx+1)
                        if FR_type==0: 
                            #Horizontal Line for Baselines
                            Purity_baseline_list = [Purity_baseline]*len(nodes_rest)
                            plt.plot(Purity_baseline_list, label = 'Louv_baseline')
                    
                        Purity_Louv_twice_list = [Purity_twice]*len(nodes_rest)
                        plt.plot(Purity_Louv_twice_list, label = '2*Louv|'+str(FR_tp_short[FR_type]))
                        plt.plot(Purity_List, label = str(FR_tp[FR_type]))
                        with open(file_path, 'a') as f:
                            f.write('Purity_Louvain_twice: ' + str(round(Purity_twice,3)) + '\n')
                            f.write('Purity after top ' + str(k*100) + '% nodes: ' + str(round(Purity_List[0],3)) + '\n')
                            f.write('Max Purity: ' + str(round(max(Purity_List),3)) + '\n')

                        #plot InEdge Degrees
                        plt.figure(InEdge_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(InEdge_List, label = str(FR_tp[FR_type]))
                    
                        
                plt.figure(nmi_fig)
                plt.tight_layout()
                plt.subplot(3, 2, idx+1)    
                #Plot all at once
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                plt.legend()

                plt.figure(purity_fig)
                plt.tight_layout()
                plt.subplot(3, 2, idx+1)
                #Plot all at once
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Purity')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                plt.legend()

                plt.figure(InEdge_fig)
                plt.tight_layout()
                plt.subplot(3, 2, idx+1)
                #Plot all at once
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('InEdge Degrees')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                plt.legend()
    
    
    p = PdfPages(pdf_name)            
    fig_nums = plt.get_fignums() 
    figs = [plt.figure(n) for n in fig_nums] 
	
	# iterating over the numbers in list 
    for fig in figs: 
		# and saving the files 
        fig.savefig(p, format='pdf') 
	
	# close the object 
    p.close() 
    plt.close('all')
    
        
            

                                
                    

                    
                    



[0 1 2 3 4 5 6 7 8 9]
(11235, 19939)
selected vertices  11235
(11235,) Counter({'TCGA-BRCA': 1226, 'TCGA-KIRC': 610, 'TCGA-LUAD': 589, 'TCGA-UCEC': 585, 'TCGA-THCA': 572, 'TCGA-HNSC': 566, 'TCGA-PRAD': 554, 'TCGA-LUSC': 552, 'TCGA-LGG': 534, 'TCGA-COAD': 514, 'TCGA-SKCM': 473, 'TCGA-STAD': 448, 'TCGA-OV': 429, 'TCGA-BLCA': 428, 'TCGA-LIHC': 424, 'TCGA-KIRP': 323, 'TCGA-CESC': 309, 'TCGA-SARC': 265, 'TCGA-ESCA': 198, 'TCGA-PCPG': 187, 'TCGA-PAAD': 183, 'TCGA-READ': 177, 'TCGA-GBM': 174, 'TCGA-TGCT': 156, 'TCGA-LAML': 151, 'TCGA-THYM': 122, 'TCGA-KICH': 91, 'TCGA-MESO': 87, 'TCGA-UVM': 80, 'TCGA-ACC': 79, 'TCGA-UCS': 57, 'TCGA-DLBC': 48, 'TCGA-CHOL': 44})
33 32
Number of nodes: 11235
 --- mRNA (top 20.0%): FL + log(n) hops + Survive: 0 | res:  0.05
NMI baseline: 0.783
NMI Louvain twice: 0.784
Number of nodes not included:  753
NMI after top  20.0 % nodes: 0.89
Max NMI:  0.892
 --- mRNA (top 20.0%): FL_ng + log(n) hops + Survive: 0 | res:  0.05
NMI baseline: 0.783
NMI Louvain twice: 0.785

C:\Users\MSI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\sparse\_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


20 19
Number of nodes: 10909
 --- 20NewsGroups_tfdif (top 20.0%): FL + log(n) hops | res:  0.05
NMI baseline: 0.293
NMI Louvain twice: 0.298
Number of nodes not included:  5586
NMI after top  20.0 % nodes: 0.578
Max NMI:  0.578
 --- 20NewsGroups_tfdif (top 20.0%): FL_ng + log(n) hops | res:  0.05
NMI baseline: 0.293
NMI Louvain twice: 0.298
Number of nodes not included:  5885
NMI after top  20.0 % nodes: 0.595
Max NMI:  0.595
 --- 20NewsGroups_tfdif (top 20.0%): FL_ng_prop + log(n) hops | res:  0.05
NMI baseline: 0.293
NMI Louvain twice: 0.301
Number of nodes not included:  6125
NMI after top  20.0 % nodes: 0.597
Max NMI:  0.597
 --- 20NewsGroups_tfdif (top 20.0%): PageRank | res:  0.05
NMI baseline: 0.293
NMI Louvain twice: 0.304
Number of nodes not included:  5257
NMI after top  20.0 % nodes: 0.49
Max NMI:  0.49
 --- 20NewsGroups_tfdif (top 20.0%): FL + log(n) hops | res:  0.25
NMI baseline: 0.298
NMI Louvain twice: 0.299
Number of nodes not included:  5586
NMI after top  20.0 % node

i:\내 드라이브\backup\document\USC\Research\MCPC\Code\Codes\New_Cleaned\FlowRank_General.py:424: RuntimeWarning: invalid value encountered in scalar divide
  rank[v]=v_cover[v]/sc


 --- Cora full (top 20.0%): FL + log(n) hops | res:  0.05
NMI baseline: 0.188
NMI Louvain twice: 0.189
Number of nodes not included:  15658
NMI after top  20.0 % nodes: 0.644
Max NMI:  0.644
 --- Cora full (top 20.0%): FL_ng + log(n) hops | res:  0.05
NMI baseline: 0.188
NMI Louvain twice: 0.005
Number of nodes not included:  16114
NMI after top  20.0 % nodes: 0.352
Max NMI:  0.352
 --- Cora full (top 20.0%): FL_ng_prop + log(n) hops | res:  0.05
NMI baseline: 0.188
NMI Louvain twice: 0.012
Number of nodes not included:  16136
NMI after top  20.0 % nodes: 0.335
Max NMI:  0.335
 --- Cora full (top 20.0%): PageRank | res:  0.05
NMI baseline: 0.188
NMI Louvain twice: 0.186
Number of nodes not included:  12445
NMI after top  20.0 % nodes: 0.643
Max NMI:  0.643
 --- Cora full (top 20.0%): FL + log(n) hops | res:  0.25
NMI baseline: 0.34
NMI Louvain twice: 0.339
Number of nodes not included:  15658
NMI after top  20.0 % nodes: 0.644
Max NMI:  0.644
 --- Cora full (top 20.0%): FL_ng + log(n) 

i:\내 드라이브\backup\document\USC\Research\MCPC\Code\Codes\New_Cleaned\FlowRank_General.py:424: RuntimeWarning: invalid value encountered in scalar divide
  rank[v]=v_cover[v]/sc


 --- Citeseer (top 20.0%): FL + log(n) hops | res:  0.05
NMI baseline: 0.382
NMI Louvain twice: 0.368
Number of nodes not included:  740
NMI after top  20.0 % nodes: 0.453
Max NMI:  0.453
 --- Citeseer (top 20.0%): FL_ng + log(n) hops | res:  0.05
NMI baseline: 0.382
NMI Louvain twice: 0.263
Number of nodes not included:  1605
NMI after top  20.0 % nodes: 0.428
Max NMI:  0.428
 --- Citeseer (top 20.0%): FL_ng_prop + log(n) hops | res:  0.05
NMI baseline: 0.382
NMI Louvain twice: 0.266
Number of nodes not included:  1597
NMI after top  20.0 % nodes: 0.425
Max NMI:  0.425
 --- Citeseer (top 20.0%): PageRank | res:  0.05
NMI baseline: 0.382
NMI Louvain twice: 0.344
Number of nodes not included:  502
NMI after top  20.0 % nodes: 0.453
Max NMI:  0.453
 --- Citeseer (top 20.0%): FL + log(n) hops | res:  0.25
NMI baseline: 0.373
NMI Louvain twice: 0.338
Number of nodes not included:  740
NMI after top  20.0 % nodes: 0.453
Max NMI:  0.453
 --- Citeseer (top 20.0%): FL_ng + log(n) hops | res:  